# Opioid data from ARCOS

This notebook takes zipped state files from The Washington Post site, opens them up one by one, and calculates rates of pill sales per pharmacy based on two methodologies. 

A csv file with population data is included in this repository. To run the notebook, a user should place all zipped ARCOS files in a subdirectory called "data". 

In [103]:
import pandas as pd
import zipfile
import gzip
import os, os.path
import shutil
pd.set_option('display.max_columns', 500)


In [182]:
states = ['AK',
'AL',
'AR',
'AZ',
'CO',
'CT',
'DE',
'GA',
'HI',
'IA',
'ID',
'IL',
'IN',
'KS',
'KY',
'LA',
'MA',
'MD',
'ME',
'MI',
'MN',
'MO',
'MS',
'MT',
'NC',
'ND',
'NE',
'NH',
'NJ',
'NM',
'NV',
'NY',
'OH',
'OK',
'OR',
'PA',
'RI',
'SC',
'SD',
'TN',
'UT',
'VA',
'VT',
'WA',
'WI',
'WV',
'WY',
'TX',
'CA',
'FL']

In [187]:
def process_one_state(state):
    # Get the raw data
    try:
        opioids = pd.read_csv('data/'+state+'/arcos-' + state.lower() + '-statewide-itemized.tsv', sep='\t', header=0)
    except:
        opioids = pd.read_csv('data/arcos-' + state.lower() + '-statewide-itemized.tsv', sep='\t', header=0)

    # Isolate pharmacies
    pharmacies = opioids[opioids.BUYER_BUS_ACT.str.contains('PHARMACY', na=False)]
    
    # ---------------------------
    # Washington Post analysis
    # ---------------------------
    
    # Calculate sales per store, by county
    per_store_by_county = pharmacies.groupby(by=['BUYER_DEA_NO','BUYER_NAME','BUYER_ADDRESS1','BUYER_CITY','BUYER_ZIP','BUYER_COUNTY','BUYER_STATE']).DOSAGE_UNIT.sum().reset_index()

    # Get population data and attach it to sales per store by county
    population = pd.read_csv('county_pop_20062012.csv')
    pos = population.columns.get_loc('variable') + 1
    population['Average_population'] = population.iloc[:, pos:].mean(axis=1)
    population.county_name = population.county_name.str.upper()
    population = population[['BUYER_STATE','BUYER_COUNTY','Average_population']]
    per_store_by_county = pd.merge(per_store_by_county,population,on = ['BUYER_STATE','BUYER_COUNTY'],how='left')
    
    # Calculate pills per total county population per year
    per_store_by_county['Rate'] = (per_store_by_county.DOSAGE_UNIT.astype(float)/per_store_by_county.Average_population.astype(float))/7
    per_store_by_county.to_csv('all_stores_' + state + '.csv')
    
    # ---------------------------
    # Alternative analysis
    # ---------------------------

    # Make clean Year variable
    pharmacies['Year'] = pharmacies.TRANSACTION_DATE
    pharmacies = pharmacies[pharmacies.Year.isnull()==False] # Maryland had one transaction with null date
    pharmacies.Year = pharmacies.Year.apply(str)
    pharmacies.Year = pharmacies.Year.str.replace(r'\.0$','') # Some files come in with decimal point in date
    pharmacies.Year = pharmacies.Year.str.strip()
    pharmacies.Year = pharmacies.Year.str[-4:]
    pharmacies.Year = pharmacies.Year.astype(int)

    # Calculate store counts and sales per store, by county, by year
    per_store_by_county_by_year = pharmacies.groupby(by=['BUYER_DEA_NO','BUYER_NAME','BUYER_ADDRESS1','BUYER_CITY','BUYER_ZIP','BUYER_COUNTY','BUYER_STATE','Year']).DOSAGE_UNIT.sum().reset_index()
    store_count_by_county_by_year = per_store_by_county_by_year.groupby(by=['BUYER_STATE','BUYER_COUNTY','Year']).BUYER_DEA_NO.count().reset_index()
    store_count_by_county_by_year = store_count_by_county_by_year.rename(columns={'BUYER_DEA_NO':'Stores_in_county'})
    per_store_by_county_by_year = pd.merge(per_store_by_county_by_year,store_count_by_county_by_year,on=['BUYER_STATE','BUYER_COUNTY','Year'],how='left')
    average_store_count_by_county = store_count_by_county_by_year.groupby(by=['BUYER_STATE','BUYER_COUNTY']).Stores_in_county.mean().reset_index()
    average_store_count_by_county = average_store_count_by_county.rename(columns={'Stores_in_county':'Average_stores_in_county'})

    # Clean population info
    population = pd.read_csv('county_pop_20062012.csv')
    population.county_name = population.county_name.str.upper()
    population = population[['BUYER_STATE','BUYER_COUNTY','pop2006','pop2007','pop2008','pop2009','pop2010','pop2011','pop2012']]
    population = pd.melt(population,id_vars=['BUYER_STATE','BUYER_COUNTY'],var_name='Year', value_name='Population')
    population.Year = population.Year.str.strip('pop')
    population.Year = population.Year.astype(int)
    per_store_by_county_by_year = pd.merge(per_store_by_county_by_year,population,on=['BUYER_STATE','BUYER_COUNTY','Year'],how='left')
        
    # Calculate pills per store share of county population in each year, then average across years at each store
    per_store_by_county_by_year['One_year_rate'] = per_store_by_county_by_year.DOSAGE_UNIT.astype(float)/(per_store_by_county_by_year.Population/per_store_by_county_by_year.Stores_in_county)
    per_store_alt = per_store_by_county_by_year.groupby(by=['BUYER_DEA_NO','BUYER_NAME','BUYER_ADDRESS1','BUYER_CITY','BUYER_ZIP','BUYER_COUNTY','BUYER_STATE']).One_year_rate.mean().reset_index()
    per_store_alt = per_store_alt.rename(columns={'One_year_rate':'Alt_rate'})
    per_store_alt = pd.merge(per_store_alt,average_store_count_by_county,on='BUYER_COUNTY',how='left')
    per_store_alt.to_csv('all_stores_alternative_rate_'+state+'.csv')


In [184]:
# Loop through the states. Unzip, process, and clear out one file at a time.
# All data should go in a subdirectory called "data"
# We will store the output in separate state csv's, concacatenating them afterward

for state in states:
    print state
    zfile  = 'data/arcos-'+state.lower()+'-statewide-itemized.tsv.zip'
    gzfile = 'data/arcos-'+state.lower()+'-statewide-itemized.tsv.gz'
    try:
        unzipped = 'data/' + state
        zip = zipfile.ZipFile(zfile, 'r')
        type = 'zip'
        zip.extractall(unzipped)
    except:
        with gzip.open(gzfile, 'r') as f_in:
            unzipped = 'data/arcos-'+state.lower()+'-statewide-itemized.tsv'
            f_out = open(unzipped, 'w+') 
            shutil.copyfileobj(f_in, f_out)
            f_in.close()
        type = 'gzip'
    process_one_state(state)
    if type == 'zip':
        shutil.rmtree(unzipped)
    elif type == 'gzip':
        os.remove(unzipped)


AK


/Users/ssuo1/OneDrive - Advance Local Media LLC/Code/opioids/ENV/lib/python2.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


AL
AR
AZ
CO
CT
DE
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
UT
VA
VT
WA
WI
WV
WY
TX
CA
FL


In [192]:
# Make single file of many
counties_full      = pd.DataFrame()
counties_full_alt  = pd.DataFrame()
for state in states:
    thisfile = pd.read_csv('all_stores_' + state + '.csv')
    counties_full = pd.concat([counties_full,thisfile])
    thisfile = pd.read_csv('all_stores_alternative_rate_' + state + '.csv')
    counties_full_alt = pd.concat([counties_full_alt,thisfile])

counties_full.to_csv('all_stores.csv')
counties_full_alt = counties_full_alt[['BUYER_DEA_NO','Alt_rate','Average_stores_in_county']]
counties_full_alt.to_csv('all_stores_alt.csv')


In [193]:
# Join Post results with alt results
counties_full = pd.merge(counties_full,counties_full_alt,on='BUYER_DEA_NO',how='left')
# Export output
counties_full.to_csv('all_stores_combo.csv')
